# E2B Unified Mount Directory Test

**✅ E2B Integration with Unified Mount Structure**

## 🎯 Purpose
Test the **universal mount directory structure** across all environments:
- **Local**: Uses `S3_MOUNT_DIR` environment variable (default: `/opt/sentient`)
- **Docker**: Uses `S3_MOUNT_DIR` environment variable (default: `/opt/sentient`)
- **E2B**: Uses `S3_MOUNT_DIR` environment variable (default: `/opt/sentient`)

## 📁 Expected Structure
```
/opt/sentient/
├── {project_id}/
│   ├── binance_toolkit/     # Binance API data
│   ├── coingecko_toolkit/   # CoinGecko API data  
│   ├── defillama_toolkit/   # DefiLlama API data
│   ├── arkham_toolkit/      # Arkham API data
│   └── results/             # AI analysis outputs
└── other_projects/
```

## 🔄 Test Flow
1. 🏠 **Local Data Fetching**: Agent with data toolkit fetches crypto data to mount directory
2. ☁️ **S3 Sync**: Data automatically syncs to S3 bucket via mount
3. 🚀 **E2B Execution**: Code runs in sandbox with same mount path `/opt/sentient`
4. 🧮 **Analysis**: Sandbox analyzes data and saves results to mount
5. 📊 **Results**: Results visible across all environments

## ✅ What's Working
- Universal `/opt/sentient` mount path across environments
- S3 bucket mounted with goofys ✅ **WORKING!**
- Cross-environment data persistence ✅

## 📋 Environment Setup

In [1]:
# Clean imports and setup
import sys
import os
import asyncio
import json
import gc
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Clean any cached modules
modules_to_remove = [m for m in sys.modules.keys() if 'sentient' in m.lower()]
for module in modules_to_remove:
    if module in sys.modules:
        del sys.modules[module]

gc.collect()
print("🔄 Environment cleaned and ready")

🔄 Environment cleaned and ready


## 🔧 Dependencies and Configuration Check

In [6]:
# Load environment variables
load_dotenv()

# Check required environment variables
required_env_vars = {
    'E2B_API_KEY': 'E2B API key for sandbox access',
    'S3_BUCKET_NAME': 'S3 bucket name for data storage',
    'S3_MOUNT_DIR': 'S3 mount directory path (default: /sentient)',
}

optional_env_vars = {
    'E2B_TEMPLATE_ID': 'Custom E2B template (defaults to sentient-e2b-s3)',
    'CURRENT_PROJECT_ID': 'Current project ID for testing',
}

print("🔍 Environment Configuration Check:")
print("=" * 50)

# Check required variables
missing_required = []
for var, description in required_env_vars.items():
    value = os.getenv(var)
    if value:
        if var in ['E2B_API_KEY']:
            masked_value = f"{value[:8]}...{value[-4:]}"
        else:
            masked_value = value
        print(f"✅ {var}: {masked_value}")
    else:
        print(f"❌ {var}: MISSING - {description}")
        missing_required.append(var)

print("\n📋 Optional Configuration:")
for var, description in optional_env_vars.items():
    value = os.getenv(var)
    if value:
        print(f"✅ {var}: {value}")
    else:
        print(f"ℹ️  {var}: Using default - {description}")


mount_dir = os.getenv("S3_MOUNT_DIR", "/opt/sentient")


if missing_required:
    print(f"\n⚠️  Missing required environment variables: {', '.join(missing_required)}")
    print("   Please update your .env file with the missing values.")
else:
    print("\n🎉 All required environment variables are configured!")

🔍 Environment Configuration Check:
✅ E2B_API_KEY: e2b_d5d5...bd00
✅ S3_BUCKET_NAME: roma-shared
✅ S3_MOUNT_DIR: /opt/sentient

📋 Optional Configuration:
ℹ️  E2B_TEMPLATE_ID: Using default - Custom E2B template (defaults to sentient-e2b-s3)
ℹ️  CURRENT_PROJECT_ID: Using default - Current project ID for testing

🎉 All required environment variables are configured!


## 📦 Import Dependencies

In [ ]:
# Import E2B and agno dependencies
try:
    from agno.tools.e2b import E2BTools
    print("✅ E2BTools imported successfully")
except ImportError as e:
    print(f"❌ Failed to import E2BTools: {e}")
    print("   Please install agno: pip install agno-ai")

# Import other standard libraries
import pandas as pd
import numpy as np
from pathlib import Path



print("✅ All dependencies imported successfully")

✅ E2BTools imported successfully
✅ All dependencies imported successfully


In [7]:
import json
import re
from typing import Any, Union

def decode_e2b_output(raw_result: Any) -> str:
    """
    Decode AgnoAgent E2BTools output properly.
    
    The AgnoAgent E2BTools.run_python_code() method returns JSON-encoded output 
    in the format: ["Logs:\nLogs(stdout: [...], stderr: [...])"]
    This function properly decodes the Unicode escapes and extracts the stdout content.
    
    Args:
        raw_result: Raw output from E2BTools.run_python_code()
        
    Returns:
        Properly decoded and formatted output string
    """
    try:
        # Handle different input types
        if isinstance(raw_result, list) and len(raw_result) > 0:
            content = raw_result[0]
        elif isinstance(raw_result, str):
            content = raw_result
        else:
            return str(raw_result)
        
        # Handle JSON-encoded list format: ["Logs:\nLogs(stdout: [...], stderr: [...])"]
        if isinstance(content, str) and content.startswith('["') and content.endswith('"]'):
            try:
                parsed_list = json.loads(content)
                if isinstance(parsed_list, list) and len(parsed_list) > 0:
                    content = parsed_list[0]
            except json.JSONDecodeError:
                pass
        
        # Extract stdout content from Logs format: "Logs:\nLogs(stdout: [...], stderr: [...])"
        if isinstance(content, str) and "Logs(stdout:" in content:
            # Use regex to extract stdout content
            stdout_pattern = r"stdout:\s*\[(.*?)\],?\s*stderr:"
            match = re.search(stdout_pattern, content, re.DOTALL)
            
            if match:
                stdout_raw = match.group(1)
                
                # Parse the stdout array content - handle quoted strings with proper escaping
                stdout_lines = []
                
                # Split by ', ' but handle embedded quotes and escapes
                current_part = ""
                in_quotes = False
                escaped = False
                
                i = 0
                while i < len(stdout_raw):
                    char = stdout_raw[i]
                    
                    if escaped:
                        current_part += char
                        escaped = False
                    elif char == '\\':
                        current_part += char
                        escaped = True
                    elif char == "'" and not escaped:
                        in_quotes = not in_quotes
                        current_part += char
                    elif char == ',' and not in_quotes and i + 1 < len(stdout_raw) and stdout_raw[i + 1] == ' ':
                        # Found separator
                        if current_part.strip():
                            stdout_lines.append(current_part.strip())
                        current_part = ""
                        i += 1  # Skip the space after comma
                    else:
                        current_part += char
                    
                    i += 1
                
                # Add the last part
                if current_part.strip():
                    stdout_lines.append(current_part.strip())
                
                # Clean and decode each part
                decoded_lines = []
                for part in stdout_lines:
                    # Remove outer quotes
                    if (part.startswith("'") and part.endswith("'")) or (part.startswith('"') and part.endswith('"')):
                        part = part[1:-1]
                    
                    # Decode JSON escapes properly
                    try:
                        # Handle JSON string encoding with proper Unicode support
                        decoded_part = json.loads(f'"{part}"')
                        decoded_lines.append(decoded_part)
                    except json.JSONDecodeError:
                        # Fallback: manual decode common escapes
                        decoded_part = part
                        decoded_part = decoded_part.replace('\\n', '\n')
                        decoded_part = decoded_part.replace('\\t', '\t')
                        decoded_part = decoded_part.replace('\\"', '"')
                        decoded_part = decoded_part.replace("\\'", "'")
                        # Handle Unicode escapes manually
                        decoded_part = decode_unicode_escapes(decoded_part)
                        decoded_lines.append(decoded_part)
                
                # Join all stdout lines
                full_output = ''.join(decoded_lines)
                return full_output
        
        # If not in Logs format, try direct JSON decode
        if isinstance(content, str):
            try:
                decoded = json.loads(f'"{content}"')
                return decode_unicode_escapes(decoded)
            except json.JSONDecodeError:
                return decode_unicode_escapes(content)
        
        return str(content)
        
    except Exception as e:
        return f"[Error decoding E2B output: {e}]\n\nRaw output:\n{str(raw_result)}"


def decode_unicode_escapes(text: str) -> str:
    """
    Decode Unicode escape sequences in text.
    
    Args:
        text: Text that may contain Unicode escapes like \\ud83c\\udfd7\\ufe0f
        
    Returns:
        Text with Unicode escapes properly decoded
    """
    if not isinstance(text, str):
        return str(text)
    
    try:
        # Handle Unicode escapes using encode/decode
        return text.encode().decode('unicode_escape')
    except (UnicodeDecodeError, UnicodeEncodeError):
        # Fallback: manual decode using codecs
        try:
            import codecs
            return codecs.decode(text, 'unicode_escape')
        except:
            # Final fallback: return as-is
            return text

print("🔧 E2B output decoder functions loaded")

🔧 E2B output decoder functions loaded


In [8]:
async def test_mount_directory():
    """Test the unified mount directory in E2B sandbox."""
    print("🏗️ MOUNT DIRECTORY VERIFICATION TEST")
    print("=" * 45)
    
    try:
        # Configuration - Use environment variable for mount directory
        template_name = os.getenv("E2B_TEMPLATE_ID", "sentient-e2b-s3")
        
        print(f"🎯 Template: {template_name}")
        print(f"📁 Mount directory (from S3_MOUNT_DIR): {mount_dir}")
        
        # Initialize E2B
        e2b_toolkit = E2BTools(
            timeout=300,
            sandbox_options={"template": template_name}
        )
        print("✅ E2B toolkit initialized")
        
        # Test mount directory and explore available directories
        mount_test_code = f"""
import os
import platform

print("🏗️ E2B MOUNT VERIFICATION")
print("=" * 30)
print(f"Platform: {{platform.platform()}}")
print(f"Python: {{platform.python_version()}}")
print(f"Working dir: {{os.getcwd()}}")

# Get mount directory from environment variable in E2B sandbox
MOUNT_DIR = os.getenv("S3_MOUNT_DIR", "/opt/sentient")
print(f"\\n📁 Checking mount directory: {{MOUNT_DIR}}")
print(f"   (S3_MOUNT_DIR env var: {{os.getenv('S3_MOUNT_DIR', 'not set')}})")

if os.path.exists(MOUNT_DIR):
    print("✅ Mount directory exists")
    
    try:
        contents = os.listdir(MOUNT_DIR)
        print(f"✅ Mount directory readable: {{len(contents)}} items")
        
        # Show contents
        if contents:
            print("📋 Current contents:")
            for item in contents[:10]:  # Show first 10 items
                item_path = os.path.join(MOUNT_DIR, item)
                if os.path.isdir(item_path):
                    try:
                        sub_items = len(os.listdir(item_path))
                        print(f"   📁 {{item}}: {{sub_items}} items")
                    except:
                        print(f"   📁 {{item}}: (cannot read)")
                else:
                    size = os.path.getsize(item_path)
                    print(f"   📄 {{item}}: {{size}} bytes")
            
            if len(contents) > 10:
                print(f"   ... and {{len(contents) - 10}} more items")
        else:
            print("📁 Mount directory is empty")
            
        # Test write access
        test_file = os.path.join(MOUNT_DIR, "mount_test.txt")
        try:
            with open(test_file, 'w') as f:
                f.write("Mount write test successful")
            
            if os.path.exists(test_file):
                print("✅ Mount directory is writable")
                os.remove(test_file)  # Clean up
                print("🧹 Test file cleaned up")
            else:
                print("❌ Write test file not found")
                
        except Exception as e:
            print(f"❌ Cannot write to mount directory: {{e}}")
            
    except Exception as e:
        print(f"❌ Cannot read mount directory: {{e}}")
        
else:
    print("❌ Mount directory does not exist")
    print(f"   Expected path: {{MOUNT_DIR}}")
    print("   This indicates the E2B template mount is not working properly")
    
    # Explore available directories to help with debugging
    print("\\n🔍 EXPLORING AVAILABLE DIRECTORIES:")
    
    # Check common mount points and symlinks from startup.sh
    potential_mounts = [
        "/workspace", "/home/user", "/tmp", "/mnt", "/media", "/opt",
        "/workspace/data", "/workspace/results", "/home/user/data",
        "/home/user/s3-bucket", "/workspace/s3-bucket"
    ]
    
    found_dirs = []
    for path in potential_mounts:
        if os.path.exists(path):
            found_dirs.append(path)
            try:
                contents = os.listdir(path)
                print(f"✅ {{path}}: {{len(contents)}} items")
                
                # Check if it's a symlink (from startup.sh)
                if os.path.islink(path):
                    target = os.readlink(path)
                    print(f"   🔗 Symlink points to: {{target}}")
                
                # Show contents
                if contents and len(contents) <= 5:
                    for item in contents:
                        print(f"   - {{item}}")
                elif contents:
                    print(f"   - {{contents[0]}}, {{contents[1]}} ... ({{len(contents)}} total)")
            except Exception as e:
                print(f"✅ {{path}}: exists but cannot list ({{e}})")
    
    print(f"\\n📋 Found {{len(found_dirs)}} potential directories")
    
    # Check startup script environment info
    print("\\n📋 E2B STARTUP ENVIRONMENT:")
    env_files = ["/home/user/.env-info", "/workspace/.env-info"]
    for env_file in env_files:
        if os.path.exists(env_file):
            print(f"Environment info from {{env_file}}:")
            try:
                with open(env_file, 'r') as f:
                    for line in f:
                        if not line.startswith('#') and '=' in line:
                            print(f"   {{line.strip()}}")
                break
            except Exception as e:
                print(f"   Cannot read {{env_file}}: {{e}}")
    
    # Check mount method used
    mount_method_file = "/tmp/mount-method"
    if os.path.exists(mount_method_file):
        try:
            with open(mount_method_file, 'r') as f:
                method = f.read().strip()
            print(f"\\n🔧 S3 Mount method used: {{method}}")
        except:
            print("\\n🔧 S3 Mount method: unknown")
    
    print("\\n💡 DIAGNOSIS:")
    print("1. Check if S3_MOUNT_DIR env var is properly set in E2B template")
    print("2. Verify AWS credentials were passed as build arguments")
    print("3. Check startup.sh logs for mount failures")
    print("4. Alternative: Use working symlinked path if available")

print("\\n✅ Mount verification completed")
"""
        
        result = e2b_toolkit.run_python_code(mount_test_code)
        decoded_output = decode_e2b_output(result)
        print("✅ Mount verification completed")
        print("📊 Output:")
        print("-" * 40)
        print(decoded_output)
        print("-" * 40)
        
        # Check if mount exists for return value
        mount_exists = "✅ Mount directory exists" in decoded_output
        
        # Extract working mount path if available
        if not mount_exists:
            lines = decoded_output.split('\n')
            for line in lines:
                # Look for working symlinks that point to S3
                if "🔗 Symlink points to:" in line and ("/data" in line or "s3" in line.lower()):
                    potential_mount = line.split("points to: ")[-1].strip()
                    print(f"\n💡 FOUND POTENTIAL WORKING MOUNT: {potential_mount}")
                    globals()['discovered_mount_dir'] = potential_mount
                    break
        
        return mount_exists
        
    except Exception as e:
        print(f"❌ Mount directory test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run the test
mount_success = await test_mount_directory()
print(f"\n🎯 Mount test result: {'✅ PASSED' if mount_success else '❌ FAILED'}")

if not mount_success:
    print("\n🔧 TROUBLESHOOTING GUIDE:")
    print("=" * 40)
    print("The S3 mount directory is not working properly.")
    print()
    print("📋 Root cause: E2B template was built without AWS credentials")
    print("🔧 Solution: Rebuild E2B template with proper build arguments:")
    print()
    print("cd docker/e2b-sandbox")
    print("e2b template build \\")
    print("    --build-arg AWS_ACCESS_KEY_ID='your_key' \\")
    print("    --build-arg AWS_SECRET_ACCESS_KEY='your_secret' \\")
    print("    --build-arg S3_BUCKET_NAME='your_bucket' \\")
    print("    --build-arg S3_MOUNT_DIR='/opt/sentient' \\")
    print("    --name sentient-e2b-s3 --force")
    print()
    print("Or run: ./setup.sh --e2b-only")
    
    if 'discovered_mount_dir' in globals():
        print(f"\n💡 TEMPORARY WORKAROUND:")
        print(f"Update .env with: S3_MOUNT_DIR={globals()['discovered_mount_dir']}")
else:
    print("\n🎉 SUCCESS: Mount directory is working correctly!")
    print(f"📁 Using mount path: {mount_dir}")
    globals()['working_mount_dir'] = mount_dir

🏗️ MOUNT DIRECTORY VERIFICATION TEST
🎯 Template: sentient-e2b-s3
📁 Mount directory (from S3_MOUNT_DIR): /opt/sentient
✅ E2B toolkit initialized
✅ Mount verification completed
📊 Output:
----------------------------------------
🏗️ E2B MOUNT VERIFICATION
Platform: Linux-6.1.102-x86_64-with-glibc2.41
Python: 3.12.11
Working dir: /home/user

📁 Checking mount directory: /opt/sentient
   (S3_MOUNT_DIR env var: /opt/sentient)
✅ Mount directory exists
✅ Mount directory readable: 2 items
📋 Current contents:
   📁 shared: 0 items
   📁 {shared}: 0 items
✅ Mount directory is writable
🧹 Test file cleaned up

✅ Mount verification completed

----------------------------------------

🎯 Mount test result: ✅ PASSED

🎉 SUCCESS: Mount directory is working correctly!
📁 Using mount path: /opt/sentient


## 📁 Test 2: Project Structure Creation

In [13]:
async def test_project_structure():
    """Test creating the expected project structure in mount directory."""
    print("📁 PROJECT STRUCTURE CREATION TEST")
    print("=" * 45)
    
    try:
        # Configuration - Use environment variable for mount directory
        template_name = os.getenv("E2B_TEMPLATE_ID", "sentient-e2b-s3")
        mount_dir = os.getenv("S3_MOUNT_DIR", "/sentient")
        
        test_project_id = f"test_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        print(f"🧪 Test Project ID: {test_project_id}")
        print(f"📁 Mount Directory (from S3_MOUNT_DIR): {mount_dir}")
        
        # Use discovered working mount if available from previous test
        if 'discovered_mount_dir' in globals():
            working_mount = globals()['discovered_mount_dir']
            print(f"🔄 Using discovered working mount: {working_mount}")
        elif 'working_mount_dir' in globals():
            working_mount = globals()['working_mount_dir']
            print(f"✅ Using confirmed working mount: {working_mount}")
        else:
            working_mount = mount_dir
            print(f"🎯 Using configured mount: {working_mount}")
        
        # Initialize E2B
        e2b_toolkit = E2BTools(
            timeout=300,
            sandbox_options={"template": template_name}
        )
        
        # Create project structure
        structure_test_code = f"""
import os
import json
from datetime import datetime

# Use environment variable or fallback to discovered mount
MOUNT_DIR = os.getenv("S3_MOUNT_DIR", "/sentient")
PROJECT_ID = "{test_project_id}"

print("📁 CREATING PROJECT STRUCTURE")
print("=" * 35)
print(f"Project: {{PROJECT_ID}}")
print(f"Mount: {{MOUNT_DIR}}")
print(f"S3_MOUNT_DIR env var: {{os.getenv('S3_MOUNT_DIR', 'not set')}}")

# Check if mount directory is available (could be symlinked)
if not os.path.exists(MOUNT_DIR):
    print(f"❌ Mount directory not available: {{MOUNT_DIR}}")
    
    # Try alternative paths from startup.sh symlinks
    alternatives = ["/workspace/data", "/home/user/data"]
    for alt in alternatives:
        if os.path.exists(alt):
            print(f"✅ Using alternative mount: {{alt}}")
            MOUNT_DIR = alt
            break
    else:
        print("❌ No alternative mount directories found")
        exit(1)

# Create project directory
project_dir = os.path.join(MOUNT_DIR, PROJECT_ID)
os.makedirs(project_dir, exist_ok=True)
print(f"✅ Created project directory: {{project_dir}}")

# Define expected toolkit directories
toolkit_dirs = [
    "binance_toolkit",
    "coingecko_toolkit",
    "defillama_toolkit", 
    "arkham_toolkit",
    "results"
]

print(f"\\n🗂️  Creating {{len(toolkit_dirs)}} directories...")

created_count = 0
for toolkit_name in toolkit_dirs:
    toolkit_dir = os.path.join(project_dir, toolkit_name)
    
    try:
        os.makedirs(toolkit_dir, exist_ok=True)
        
        # Create a test file in each directory
        test_file = os.path.join(toolkit_dir, "structure_test.json")
        test_data = {{
            "toolkit": toolkit_name,
            "project_id": PROJECT_ID,
            "created_at": datetime.now().isoformat(),
            "environment": "e2b_sandbox",
            "mount_path": MOUNT_DIR,
            "env_s3_mount_dir": os.getenv('S3_MOUNT_DIR', 'not_set'),
            "test_type": "structure_creation"
        }}
        
        with open(test_file, 'w') as f:
            json.dump(test_data, f, indent=2)
        
        file_size = os.path.getsize(test_file)
        print(f"✅ {{toolkit_name}}: created directory + test file ({{file_size}} bytes)")
        created_count += 1
        
    except Exception as e:
        print(f"❌ {{toolkit_name}}: failed to create - {{e}}")

# Summary
print(f"\\n📊 Structure Creation Summary:")
print(f"   Final mount path used: {{MOUNT_DIR}}")
print(f"   Project directory: {{project_dir}}")
print(f"   Toolkit directories created: {{created_count}}/{{len(toolkit_dirs)}}")

# Verify structure
if os.path.exists(project_dir):
    contents = os.listdir(project_dir)
    print(f"\\n📋 Final structure verification:")
    for item in sorted(contents):
        item_path = os.path.join(project_dir, item)
        if os.path.isdir(item_path):
            files = os.listdir(item_path)
            print(f"   📁 {{item}}: {{len(files)}} files")
            for f in files:
                f_path = os.path.join(item_path, f)
                f_size = os.path.getsize(f_path)
                print(f"      📄 {{f}} ({{f_size}} bytes)")

if created_count == len(toolkit_dirs):
    print("\\n🎉 Project structure created successfully!")
    print("✅ All expected directories and test files are present")
    print("🔄 This structure should be visible across all environments")
    print(f"🌐 S3 sync: Files should appear in S3 bucket via {{MOUNT_DIR}} mount")
else:
    print(f"\\n⚠️  Partial success: {{created_count}}/{{len(toolkit_dirs)}} directories created")

print("\\n✅ Structure creation test completed")
"""
        
        result = e2b_toolkit.run_python_code(structure_test_code)
        decoded_output = decode_e2b_output(result)
        print("✅ Structure creation completed")
        print("📊 Output:")
        print("-" * 40)
        print(decoded_output)
        print("-" * 40)
        
        # Save test project ID for later tests
        globals()['test_project_id'] = test_project_id
        
        # Check if structure was created successfully
        structure_success = "🎉 Project structure created successfully!" in decoded_output
        
        return structure_success
        
    except Exception as e:
        print(f"❌ Project structure test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run the test
structure_success = await test_project_structure()
print(f"\n🎯 Structure test result: {'✅ PASSED' if structure_success else '❌ FAILED'}")

📁 PROJECT STRUCTURE CREATION TEST
🧪 Test Project ID: test_20250822_214954
📁 Mount Directory (from S3_MOUNT_DIR): /opt/sentient
✅ Using confirmed working mount: /opt/sentient
✅ Structure creation completed
📊 Output:
----------------------------------------
📁 CREATING PROJECT STRUCTURE
Project: test_20250822_214954
Mount: /opt/sentient
S3_MOUNT_DIR env var: /opt/sentient
✅ Created project directory: /opt/sentient/test_20250822_214954

🗂️  Creating 5 directories...
✅ binance_toolkit: created directory + test file (261 bytes)
✅ coingecko_toolkit: created directory + test file (263 bytes)
✅ defillama_toolkit: created directory + test file (263 bytes)
✅ arkham_toolkit: created directory + test file (260 bytes)
✅ results: created directory + test file (253 bytes)

📊 Structure Creation Summary:
   Final mount path used: /opt/sentient
   Project directory: /opt/sentient/test_20250822_214954
   Toolkit directories created: 5/5

📋 Final structure verification:
   📁 arkham_toolkit: 1 files
      📄

## 💰 Test 3: Data Analysis Workflow

In [11]:
async def test_data_analysis_workflow():
    """Test a complete data analysis workflow using the mount directory."""
    print("💰 DATA ANALYSIS WORKFLOW TEST")
    print("=" * 40)
    
    try:
        # Configuration - Use environment variable for mount directory
        template_name = os.getenv("E2B_TEMPLATE_ID", "sentient-e2b-s3")
        mount_dir = os.getenv("S3_MOUNT_DIR", "/opt/sentient")
        mount_dir = os.path.expandvars(mount_dir)  # Resolve $HOME etc.
        
        # Use existing test project or create new one
        if 'test_project_id' in globals():
            project_id = globals()['test_project_id']
            print(f"🔄 Using existing project: {project_id}")
        else:
            project_id = f"analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            print(f"🆕 Creating new project: {project_id}")
        
        print(f"📁 Mount Directory (from S3_MOUNT_DIR): {mount_dir}")
        
        # Use discovered working mount if available from previous test
        if 'discovered_mount_dir' in globals():
            working_mount = globals()['discovered_mount_dir']
            print(f"🔄 Using discovered working mount: {working_mount}")
        elif 'working_mount_dir' in globals():
            working_mount = globals()['working_mount_dir']
            print(f"✅ Using confirmed working mount: {working_mount}")
        else:
            working_mount = mount_dir
            print(f"🎯 Using configured mount: {working_mount}")
        
        # Initialize E2B
        e2b_toolkit = E2BTools(
            timeout=300,
            sandbox_options={"template": template_name}
        )
        
        # Run data analysis workflow
        analysis_code = f"""
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime, timedelta

# Use environment variable or fallback to discovered mount
MOUNT_DIR = os.getenv("S3_MOUNT_DIR", "/opt/sentient")
PROJECT_ID = "{project_id}"

print("💰 DATA ANALYSIS WORKFLOW")
print("=" * 35)
print(f"Project: {{PROJECT_ID}}")
print(f"Mount: {{MOUNT_DIR}}")
print(f"S3_MOUNT_DIR env var: {{os.getenv('S3_MOUNT_DIR', 'not set')}})")

# Check if mount directory is available (could be symlinked)
if not os.path.exists(MOUNT_DIR):
    print(f"❌ Mount directory not available: {{MOUNT_DIR}}")
    
    # Try alternative paths from startup.sh symlinks
    alternatives = ["/workspace/data", "/home/user/data"]
    for alt in alternatives:
        if os.path.exists(alt):
            print(f"✅ Using alternative mount: {{alt}}")
            MOUNT_DIR = alt
            break
    else:
        print("❌ No alternative mount directories found - using /tmp")
        MOUNT_DIR = "/tmp"

# Ensure project structure exists
project_dir = os.path.join(MOUNT_DIR, PROJECT_ID)
os.makedirs(project_dir, exist_ok=True)

# Ensure results directory exists
results_dir = os.path.join(project_dir, "results")
os.makedirs(results_dir, exist_ok=True)

print(f"✅ Project directory ready: {{project_dir}}")

# Step 1: Generate sample crypto data (simulating toolkit data)
print("\\n📊 Step 1: Generating sample crypto data...")

symbols = ['BTC', 'ETH', 'SOL', 'ADA', 'DOT']
base_prices = {{
    'BTC': 45000,
    'ETH': 3000, 
    'SOL': 100,
    'ADA': 0.50,
    'DOT': 25
}}

# Generate 24 hours of hourly data
hours = 24
timestamps = [datetime.now() - timedelta(hours=i) for i in range(hours, 0, -1)]

np.random.seed(42)  # Reproducible results
crypto_data = []

for symbol in symbols:
    base_price = base_prices[symbol]
    
    for i, timestamp in enumerate(timestamps):
        # Simulate realistic price movement
        volatility = np.random.normal(0, 0.02)  # 2% volatility
        trend = 0.001 * np.sin(i / 6)  # 6-hour cycle
        price = base_price * (1 + trend + volatility)
        
        crypto_data.append({{
            'timestamp': timestamp.isoformat(),
            'symbol': symbol,
            'price': round(price, 6),
            'volume': np.random.uniform(1000000, 5000000)
        }})

df = pd.DataFrame(crypto_data)
print(f"✅ Generated {{len(df)}} data points for {{len(symbols)}} symbols")

# Step 2: Perform analysis
print("\\n🔍 Step 2: Performing market analysis...")

# Current prices
current_prices = df.groupby('symbol')['price'].last()
print("\\n💰 Current Prices:")
for symbol, price in current_prices.items():
    print(f"   {{symbol}}: ${{price:,.2f}}")

# Calculate 24h changes
first_prices = df.groupby('symbol')['price'].first()
changes_24h = ((current_prices - first_prices) / first_prices * 100)

print("\\n📈 24h Price Changes:")
for symbol, change in changes_24h.items():
    emoji = "🟢" if change > 0 else "🔴" if change < 0 else "⚪"
    print(f"   {{emoji}} {{symbol}}: {{change:+.2f}}%")

# Calculate volatility
volatility = df.groupby('symbol')['price'].std() / df.groupby('symbol')['price'].mean() * 100
print("\\n📊 Price Volatility (24h):")
for symbol, vol in volatility.items():
    print(f"   {{symbol}}: {{vol:.2f}}%")

# Step 3: Save results to mount directory
print("\\n💾 Step 3: Saving analysis results...")

# Comprehensive analysis results
analysis_results = {{
    'timestamp': datetime.now().isoformat(),
    'project_id': PROJECT_ID,
    'analysis_type': 'crypto_market_analysis',
    'data_period': '24h',
    'symbols_analyzed': len(symbols),
    'data_points': len(df),
    'mount_path_used': MOUNT_DIR,
    'env_s3_mount_dir': os.getenv('S3_MOUNT_DIR', 'not_set'),
    'current_prices': dict(current_prices),
    'price_changes_24h': dict(changes_24h),
    'volatility_24h': dict(volatility),
    'summary': {{
        'best_performer': changes_24h.idxmax(),
        'worst_performer': changes_24h.idxmin(),
        'highest_volatility': volatility.idxmax(),
        'lowest_volatility': volatility.idxmin()
    }}
}}

# Save analysis results
results_file = os.path.join(results_dir, 'market_analysis.json')
with open(results_file, 'w') as f:
    json.dump(analysis_results, f, indent=2)

results_size = os.path.getsize(results_file)
print(f"✅ Analysis results saved: {{results_file}} ({{results_size}} bytes)")

# Save raw data for each toolkit
for symbol in symbols:
    symbol_data = df[df['symbol'] == symbol].copy()
    toolkit_name = f"{{symbol.lower()}}_toolkit"  # Not exactly the real toolkit names, but for demo
    toolkit_dir = os.path.join(project_dir, toolkit_name)
    os.makedirs(toolkit_dir, exist_ok=True)
    
    # Save symbol-specific data
    symbol_file = os.path.join(toolkit_dir, f'{{symbol.lower()}}_24h_data.csv')
    symbol_data.to_csv(symbol_file, index=False)
    
    # Save symbol summary
    summary_file = os.path.join(toolkit_dir, f'{{symbol.lower()}}_summary.json')
    symbol_summary = {{
        'symbol': symbol,
        'current_price': float(current_prices[symbol]),
        'change_24h': float(changes_24h[symbol]),
        'volatility_24h': float(volatility[symbol]),
        'data_points': len(symbol_data),
        'mount_path_used': MOUNT_DIR,
        'price_range': {{
            'min': float(symbol_data['price'].min()),
            'max': float(symbol_data['price'].max()),
            'avg': float(symbol_data['price'].mean())
        }}
    }}
    
    with open(summary_file, 'w') as f:
        json.dump(symbol_summary, f, indent=2)

print(f"✅ Individual symbol data saved to toolkit directories")

# Step 4: Verify final structure
print("\\n📋 Step 4: Verifying project structure...")

if os.path.exists(project_dir):
    contents = os.listdir(project_dir)
    print(f"\\n📁 Project '{{PROJECT_ID}}' structure:")
    for item in sorted(contents):
        item_path = os.path.join(project_dir, item)
        if os.path.isdir(item_path):
            files = os.listdir(item_path)
            total_size = sum(os.path.getsize(os.path.join(item_path, f)) 
                           for f in files if os.path.isfile(os.path.join(item_path, f)))
            print(f"   📁 {{item}}: {{len(files)}} files, {{total_size}} bytes total")

print("\\n🎉 Data analysis workflow completed successfully!")
print("✅ All data saved to unified mount directory")
print("🔄 Results should now be visible across all environments")
print(f"🌐 S3 sync: Files saved to {{MOUNT_DIR}} should sync to S3 bucket")

print("\\n✅ Analysis workflow test completed")
"""
        
        result = e2b_toolkit.run_python_code(analysis_code)
        decoded_output = decode_e2b_output(result)
        print("✅ Analysis workflow completed")
        print("📊 Output:")
        print("-" * 40)
        print(decoded_output)
        print("-" * 40)
        
        # Check if analysis was successful
        analysis_success = "🎉 Data analysis workflow completed successfully!" in decoded_output
        
        return analysis_success
        
    except Exception as e:
        print(f"❌ Data analysis workflow test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run the test
analysis_success = await test_data_analysis_workflow()
print(f"\n🎯 Analysis workflow result: {'✅ PASSED' if analysis_success else '❌ FAILED'}")

💰 DATA ANALYSIS WORKFLOW TEST
🔄 Using existing project: test_20250822_012124
📁 Mount Directory (from S3_MOUNT_DIR): /opt/sentient
✅ Using confirmed working mount: /opt/sentient
✅ Analysis workflow completed
📊 Output:
----------------------------------------
💰 DATA ANALYSIS WORKFLOW
Project: test_20250822_012124
Mount: /opt/sentient
S3_MOUNT_DIR env var: /opt/sentient)
✅ Project directory ready: /opt/sentient/test_20250822_012124

📊 Step 1: Generating sample crypto data...
✅ Generated 120 data points for 5 symbols

🔍 Step 2: Performing market analysis...

💰 Current Prices:
   ADA: $0.49
   BTC: $45,148.47
   DOT: $24.36
   ETH: $2,980.15
   SOL: $100.54

📈 24h Price Changes:
   🔴 ADA: -3.69%
   🔴 BTC: -0.66%
   🔴 DOT: -6.06%
   🔴 ETH: -2.11%
   🟢 SOL: +0.98%

📊 Price Volatility (24h):
   ADA: 1.87%
   BTC: 1.71%
   DOT: 1.74%
   ETH: 1.37%
   SOL: 1.66%

💾 Step 3: Saving analysis results...
✅ Analysis results saved: /opt/sentient/test_20250822_012124/results/market_analysis.json (927 by

## 🤖 Test 4: Real Crypto Agent + E2B Analysis Workflow"

In [12]:
async def test_crypto_agent_e2b_workflow():
    """Test real crypto agent data fetching followed by E2B analysis."""
    print("🤖 CRYPTO AGENT + E2B ANALYSIS WORKFLOW")
    print("=" * 50)
    
    try:
        # Configuration - Use environment variable for mount directory
        template_name = os.getenv("E2B_TEMPLATE_ID", "sentient-e2b-s3")
        mount_dir = os.getenv("S3_MOUNT_DIR", "/sentient")
        
        project_id = f"agent_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        print(f"🧪 Project ID: {project_id}")
        print(f"📁 Mount Directory (from S3_MOUNT_DIR): {mount_dir}")
        
        # Use discovered working mount if available from previous tests
        if 'discovered_mount_dir' in globals():
            working_mount = globals()['discovered_mount_dir']
            print(f"🔄 Using discovered working mount: {working_mount}")
        elif 'working_mount_dir' in globals():
            working_mount = globals()['working_mount_dir']
            print(f"✅ Using confirmed working mount: {working_mount}")
        else:
            working_mount = mount_dir
            print(f"🎯 Using configured mount: {working_mount}")
        
        # Step 1: Initialize crypto data toolkits (real data fetching)
        print("\n📊 Step 1: Initializing real crypto data toolkits...")
        
        try:
            from sentientresearchagent.hierarchical_agent_framework.toolkits.data.binance_toolkit import BinanceToolkit
            from sentientresearchagent.hierarchical_agent_framework.toolkits.data.coingecko_toolkit import CoinGeckoToolkit
            
            # Initialize with mount directory for data persistence
            os.environ["CURRENT_PROJECT_ID"] = project_id
            os.environ["S3_MOUNT_ENABLED"] = "true"
            os.environ["S3_MOUNT_DIR"] = working_mount  # Use working mount
            
            # Initialize toolkits - they should use BaseDataToolkit and save to mount
            binance_toolkit = BinanceToolkit(
                symbols=['BTCUSDT', 'ETHUSD'],
                default_market_type="spot"
            )
            
            coingecko_toolkit = CoinGeckoToolkit()
            
            print("✅ Real crypto toolkits initialized")
            print(f"   BinanceToolkit: {binance_toolkit.__class__.__name__}")
            print(f"   CoinGeckoToolkit: {coingecko_toolkit.__class__.__name__}")
            print(f"   Configured S3_MOUNT_DIR: {working_mount}")
            
        except ImportError as e:
            print(f"⚠️  Could not import toolkits: {e}")
            print("   Using simulated data instead")
            binance_toolkit = None
            coingecko_toolkit = None
        
        # Step 2: Fetch real crypto data using toolkits
        print("\n💰 Step 2: Fetching real crypto data...")
        
        fetched_data = {}
        
        if binance_toolkit:
            try:
                # Fetch real current prices
                btc_price = await binance_toolkit.get_current_price('BTCUSDT')
                eth_price = await binance_toolkit.get_current_price('ETHUSD')
                
                # Fetch some historical data
                btc_klines = await binance_toolkit.get_klines('BTCUSDT', interval='1h', limit=24)
                
                fetched_data['binance'] = {
                    'btc_price': btc_price,
                    'eth_price': eth_price,
                    'btc_24h_data': btc_klines
                }
                
                print(f"✅ Binance data fetched:")
                print(f"   BTC Price: ${btc_price.get('price', 'N/A')}")
                print(f"   ETH Price: ${eth_price.get('price', 'N/A')}")
                print(f"   BTC 24h data points: {len(btc_klines.get('data', [])) if isinstance(btc_klines, dict) else 'N/A'}")
                
            except Exception as e:
                print(f"⚠️  Binance data fetch error: {e}")
                fetched_data['binance_error'] = str(e)
        
        if coingecko_toolkit:
            try:
                # Get market data from CoinGecko
                btc_market = await coingecko_toolkit.get_coin_data('bitcoin')
                market_overview = await coingecko_toolkit.get_global_market_data()
                
                fetched_data['coingecko'] = {
                    'btc_market': btc_market,
                    'global_market': market_overview
                }
                
                print(f"✅ CoinGecko data fetched:")
                print(f"   Bitcoin market data: Available")
                print(f"   Global market data: Available")
                
            except Exception as e:
                print(f"⚠️  CoinGecko data fetch error: {e}")
                fetched_data['coingecko_error'] = str(e)
        
        # Step 3: Initialize E2B for advanced analysis
        print("\n🚀 Step 3: Initializing E2B for advanced analysis...")
        
        from agno.tools.e2b import E2BTools
        from agno.tools.reasoning import ReasoningTools
        from agno.agent import Agent as AgnoAgent
        from agno.models.litellm import LiteLLM
        
        # Create E2B-enabled crypto analyzer agent
        crypto_analyzer_agent = AgnoAgent(
            model=LiteLLM(id="openrouter/anthropic/claude-sonnet-4"),
            tools=[
                E2BTools(sandbox_options={"template": template_name}),
                ReasoningTools()
            ],
            name="CryptoAnalyzerWithE2B",
            system_message=f"""
You are an expert cryptocurrency analyst with access to a secure E2B sandbox environment for advanced data analysis.

Key capabilities:
1. E2B sandbox for secure Python code execution
2. Access to unified mount directory for data persistence
3. Advanced data analysis libraries: pandas, numpy, matplotlib, seaborn
4. Cross-environment data sharing

IMPORTANT MOUNT DIRECTORY CONFIGURATION:
- The mount directory is configured via S3_MOUNT_DIR environment variable
- In E2B sandbox, check: os.getenv("S3_MOUNT_DIR", "/sentient")
- Use this path consistently for all file operations
- If mount directory doesn't exist, check alternative paths like /workspace/data

When analyzing data:
1. Get mount directory: MOUNT_DIR = os.getenv("S3_MOUNT_DIR", "/sentient")
2. Save all analysis results to MOUNT_DIR/{{project_id}}/results/
3. Use the E2B sandbox for all computations and visualizations
4. Create comprehensive reports with actionable insights
5. Ensure data persists across environments

Always provide thorough analysis with clear explanations and save results for future access.
""",
            show_tool_calls=True,
            markdown=True
        )
        
        print("✅ Crypto analyzer agent with E2B initialized")
        
        # Step 4: Run comprehensive analysis using both real data and E2B
        print("\n🔍 Step 4: Running comprehensive crypto analysis...")
        
        analysis_query = f"""
Perform a comprehensive cryptocurrency market analysis using the E2B sandbox environment.

PROJECT SETUP:
- Project ID: {project_id}
- Mount directory: Use S3_MOUNT_DIR environment variable (fallback: /sentient)
- Check available paths: /workspace/data, /home/user/data if primary mount fails

ANALYSIS TASKS:
1. Get mount directory from environment: MOUNT_DIR = os.getenv("S3_MOUNT_DIR", "/sentient")
2. If MOUNT_DIR doesn't exist, try alternatives: /workspace/data, /home/user/data
3. Create project directory structure in MOUNT_DIR/{project_id}/
4. Process and analyze this real market data: {json.dumps(fetched_data, indent=2, default=str)}
5. Perform technical analysis on the price data
6. Calculate key metrics: volatility, returns, moving averages
7. Generate market insights and trading signals
8. Create visualizations (save as PNG files)
9. Save comprehensive analysis report as JSON
10. Verify all files are saved to the mount directory

Deliverables:
- Save analysis report to: MOUNT_DIR/{project_id}/results/crypto_analysis_report.json
- Save processed data to: MOUNT_DIR/{project_id}/results/processed_data.csv  
- Save visualizations to: MOUNT_DIR/{project_id}/results/*.png
- Create summary file: MOUNT_DIR/{project_id}/results/analysis_summary.md

IMPORTANT: Always check if the mount directory exists first, and include debugging info about paths used.

Use the E2B sandbox for all computations and file operations. Ensure data persists in the unified mount directory.
"""
        
        print(f"🤖 Agent query (preview): {analysis_query[:200]}...")
        print("\n⏳ Running analysis (this may take 2-3 minutes)...")
        
        try:
            # Run the analysis with timeout
            analysis_response = await asyncio.wait_for(
                crypto_analyzer_agent.arun(analysis_query),
                timeout=300  # 5 minute timeout
            )
            
            print("✅ Analysis completed successfully!")
            print("\n📄 Agent Response:")
            print("-" * 60)
            # Truncate very long responses for readability
            if len(str(analysis_response)) > 2000:
                print(f"{str(analysis_response)[:2000]}...\n[Response truncated - full response was {len(str(analysis_response))} characters]")
            else:
                print(analysis_response)
            print("-" * 60)
            
        except asyncio.TimeoutError:
            print("⏰ Analysis timed out after 5 minutes")
            analysis_response = "TIMEOUT"
        except Exception as e:
            print(f"❌ Analysis failed: {e}")
            analysis_response = f"ERROR: {e}"
        
        # Step 5: Verify results in E2B mount directory
        print("\n🔍 Step 5: Verifying results in mount directory...")
        
        e2b_toolkit = E2BTools(sandbox_options={"template": template_name})
        
        verification_code = f"""
import os
import json
import glob
from pathlib import Path

# Get mount directory from environment variable
MOUNT_DIR = os.getenv("S3_MOUNT_DIR", "/sentient")
PROJECT_ID = "{project_id}"

print("🔍 RESULTS VERIFICATION")
print("=" * 30)
print(f"Project: {{PROJECT_ID}}")
print(f"Expected mount: {{MOUNT_DIR}}")
print(f"S3_MOUNT_DIR env var: {{os.getenv('S3_MOUNT_DIR', 'not set')}}")

# Check if mount directory exists, try alternatives if not
if not os.path.exists(MOUNT_DIR):
    print(f"❌ Primary mount directory not found: {{MOUNT_DIR}}")
    
    # Try alternative paths from startup.sh symlinks
    alternatives = ["/workspace/data", "/home/user/data"]
    for alt in alternatives:
        if os.path.exists(alt):
            print(f"✅ Using alternative mount: {{alt}}")
            MOUNT_DIR = alt
            break
    else:
        print("❌ No alternative mount directories found")

# Check project directory
project_dir = os.path.join(MOUNT_DIR, PROJECT_ID)
if os.path.exists(project_dir):
    print(f"✅ Project directory exists: {{project_dir}}")
    
    # Check results directory
    results_dir = os.path.join(project_dir, "results")
    if os.path.exists(results_dir):
        print(f"✅ Results directory exists: {{results_dir}}")
        
        # List all files in results
        results_files = os.listdir(results_dir)
        print(f"\\n📋 Results files ({{len(results_files)}}):")
        
        total_size = 0
        for f in sorted(results_files):
            f_path = os.path.join(results_dir, f)
            if os.path.isfile(f_path):
                size = os.path.getsize(f_path)
                total_size += size
                print(f"   📄 {{f}}: {{size:,}} bytes")
                
                # Show content preview for key files
                if f.endswith('.json'):
                    try:
                        with open(f_path, 'r') as file:
                            data = json.load(file)
                        print(f"      🔍 JSON keys: {{list(data.keys())[:5]}}")
                    except:
                        pass
            elif os.path.isdir(f_path):
                sub_files = len(os.listdir(f_path))
                print(f"   📁 {{f}}: {{sub_files}} items")
        
        print(f"\\n📊 Total results size: {{total_size:,}} bytes")
        
        # Check for expected files
        expected_files = [
            'crypto_analysis_report.json',
            'processed_data.csv',
            'analysis_summary.md'
        ]
        
        found_files = 0
        for expected in expected_files:
            if expected in results_files:
                print(f"✅ Found expected file: {{expected}}")
                found_files += 1
            else:
                print(f"❌ Missing expected file: {{expected}}")
        
        # Check for visualizations
        png_files = [f for f in results_files if f.endswith('.png')]
        if png_files:
            print(f"✅ Found {{len(png_files)}} visualization(s): {{png_files}}")
        else:
            print("❌ No PNG visualizations found")
        
        print(f"\\n📈 Success rate: {{found_files}}/{{len(expected_files)}} expected files found")
        print(f"📁 Final mount directory used: {{MOUNT_DIR}}")
        
    else:
        print(f"❌ Results directory not found: {{results_dir}}")
else:
    print(f"❌ Project directory not found: {{project_dir}}")
    print(f"   Looking for: {{MOUNT_DIR}}/{{PROJECT_ID}}")

print("\\n✅ Verification completed")
"""
        
        verification_result = e2b_toolkit.run_python_code(verification_code)
        decoded_verification = decode_e2b_output(verification_result)
        print("✅ Verification completed")
        print("📊 Verification Output:")
        print("-" * 40)
        print(decoded_verification)
        print("-" * 40)
        
        # Success determination
        success = (
            analysis_response != "TIMEOUT" and
            "ERROR" not in str(analysis_response) and
            "✅" in str(decoded_verification)
        )
        
        print(f"\n🎯 Crypto Agent + E2B Workflow Result: {'✅ PASSED' if success else '❌ FAILED'}")
        
        if success:
            print("🎉 Complete workflow successful!")
            print(f"   📁 Real data fetched from crypto APIs")
            print(f"   🤖 Agent analysis completed in E2B sandbox")
            print(f"   💾 Results saved to unified mount directory")
            print(f"   🔄 Data accessible across all environments")
            print(f"   🌐 Using environment-variable mount: {working_mount}")
        
        return success
        
    except Exception as e:
        print(f"❌ Crypto agent + E2B workflow failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run the test
crypto_agent_success = await test_crypto_agent_e2b_workflow()
print(f"\n🎯 Final result: {'✅ PASSED' if crypto_agent_success else '❌ FAILED'}")

2025-08-22 01:29:19.450 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.base.base_api:_init_cache_system:270 - Initialized generic cache system with TTL: 3600s
2025-08-22 01:29:19.451 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.utils.http_client:__init__:96 - Initialized DataHTTPClient with 30.0s timeout
2025-08-22 01:29:19.451 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.base.base_api:_init_standard_configuration:559 - Initialized standard configuration: timeout=30.0s, retries=3, cache_ttl=3600s
2025-08-22 01:29:19.452 | INFO     | sentientresearchagent.hierarchical_agent_framework.toolkits.base.base_data:_init_data_helpers:95 - Using S3 mounted directory for binance: /opt/sentient


🤖 CRYPTO AGENT + E2B ANALYSIS WORKFLOW
🧪 Project ID: agent_test_20250822_012919
📁 Mount Directory (from S3_MOUNT_DIR): /opt/sentient
✅ Using confirmed working mount: /opt/sentient

📊 Step 1: Initializing real crypto data toolkits...


2025-08-22 01:29:20.435 | INFO     | sentientresearchagent.hierarchical_agent_framework.toolkits.base.base_data:_detect_e2b_context:131 - S3 integration detected with bucket: roma-shared
2025-08-22 01:29:20.436 | INFO     | sentientresearchagent.hierarchical_agent_framework.toolkits.base.base_data:_init_data_helpers:120 - Data helpers initialized - Project: agent_test_20250822_012919, Toolkit: binance, Dir: /opt/sentient/agent_test_20250822_012919/binance, S3: True
2025-08-22 01:29:20.437 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.data.binance_toolkit:__init__:300 - Initialized Multi-Market BinanceToolkit with default market 'spot' and 2 symbols
2025-08-22 01:29:20.438 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.data.coingecko_toolkit:__init__:319 - Using CoinGecko Pro API with API key
2025-08-22 01:29:20.438 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.base.base_api:_init_cache_system:270 - Initiali

✅ Real crypto toolkits initialized
   BinanceToolkit: BinanceToolkit
   CoinGeckoToolkit: CoinGeckoToolkit
   Configured S3_MOUNT_DIR: /opt/sentient

💰 Step 2: Fetching real crypto data...


2025-08-22 01:29:21.323 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.base.base_api:_cache_data:311 - Cached set data (246 items) for key 'symbols_spot'
2025-08-22 01:29:21.323 | INFO     | sentientresearchagent.hierarchical_agent_framework.toolkits.data.binance_toolkit:reload_symbols:509 - Loaded 246 symbols for Binance Spot Trading
2025-08-22 01:29:21.324 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.utils.http_client:_make_request:317 - Making GET request to spot/api/v3/ticker/price (attempt 1)
2025-08-22 01:29:21.444 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.utils.http_client:_make_request:317 - Making GET request to spot/api/v3/ticker/price (attempt 1)
2025-08-22 01:29:21.570 | DEBUG    | sentientresearchagent.hierarchical_agent_framework.toolkits.utils.http_client:_make_request:317 - Making GET request to spot/api/v3/klines (attempt 1)


✅ Binance data fetched:
   BTC Price: $N/A
   ETH Price: $N/A
   BTC 24h data points: 24
⚠️  CoinGecko data fetch error: 'CoinGeckoToolkit' object has no attribute 'get_coin_data'

🚀 Step 3: Initializing E2B for advanced analysis...
✅ Crypto analyzer agent with E2B initialized

🔍 Step 4: Running comprehensive crypto analysis...
🤖 Agent query (preview): 
Perform a comprehensive cryptocurrency market analysis using the E2B sandbox environment.

PROJECT SETUP:
- Project ID: agent_test_20250822_012919
- Mount directory: Use S3_MOUNT_DIR environment vari...

⏳ Running analysis (this may take 2-3 minutes)...
⏰ Analysis timed out after 5 minutes

🔍 Step 5: Verifying results in mount directory...
✅ Verification completed
📊 Verification Output:
----------------------------------------
🔍 RESULTS VERIFICATION
Project: agent_test_20250822_012919
Expected mount: /opt/sentient
S3_MOUNT_DIR env var: /opt/sentient
"â Project directory exists: /opt/sentient/agent_test_20250822_012919
â Results dire

## 📊 Test Summary

In [ ]:
def print_test_summary():
    """Print a comprehensive summary of all tests."""
    print("📊 E2B UNIFIED MOUNT TEST SUMMARY")
    print("=" * 50)
    
    # Get mount directory from environment variable
    configured_mount_dir = os.getenv("S3_MOUNT_DIR", "/data/sentient")
    configured_mount_dir = os.path.expandvars(configured_mount_dir)
    
    print(f"🎯 Configured Mount Directory (S3_MOUNT_DIR): {configured_mount_dir}")
    
    # Show discovered working mount if different
    if 'discovered_mount_dir' in globals():
        print(f"🔄 Discovered Working Mount: {globals()['discovered_mount_dir']}")
    if 'working_mount_dir' in globals():
        print(f"✅ Confirmed Working Mount: {globals()['working_mount_dir']}")
    
    # Collect test results
    tests = [
        ("Mount Directory Verification", globals().get('mount_success', False)),
        ("Project Structure Creation", globals().get('structure_success', False)),
        ("Data Analysis Workflow", globals().get('analysis_success', False)),
        ("Crypto Agent + E2B Analysis", globals().get('crypto_agent_success', False)),
    ]
    
    passed = 0
    total = 0
    
    print("\n🧪 Test Results:")
    for test_name, result in tests:
        if result is None:
            status = "⏭️  SKIPPED"
        elif result:
            status = "✅ PASSED"
            passed += 1
            total += 1
        else:
            status = "❌ FAILED"
            total += 1
        
        print(f"  {status} {test_name}")
    
    print(f"\n📈 Overall Results: {passed}/{total} tests passed")
    
    # Results analysis
    if passed == total and total > 0:
        print("\n🎉 SUCCESS: All tests passed!")
        print("✅ Unified mount directory system is working correctly")
        print("✅ Environment variable configuration (S3_MOUNT_DIR) is functional")
        print("✅ Cross-environment data persistence is operational")
        print("✅ Project structure creation and data workflows are working")
        print("✅ Real crypto agent + E2B analysis integration is functional")
    elif passed > 0:
        print(f"\n⚠️  PARTIAL SUCCESS: {passed}/{total} tests passed")
        print("   Check the failed tests above for specific issues")
    else:
        print(f"\n❌ FAILURE: No tests passed")
        print("   Check E2B template configuration and mount setup")
    
    # Key information
    print("\n💡 Key System Information:")
    print(f"  📁 Mount Directory: {configured_mount_dir} (from S3_MOUNT_DIR env var)")
    print(f"  🏗️  E2B Template: {os.getenv('E2B_TEMPLATE_ID', 'sentient-e2b-s3')}")
    print(f"  🗂️  Project Structure: MOUNT_DIR/{{project_id}}/{{toolkit_name|results}}/")
    print(f"  🔄 Data Flow: Local → S3 → E2B sandbox (consistent paths)")
    print(f"  🤖 Agent Integration: Real API data → E2B analysis → Unified results")
    print(f"  🌐 Environment Variable: All components use S3_MOUNT_DIR for consistency")
    
    # Usage guidance
    print("\n🔗 Usage Instructions:")
    print("  1. Set project ID: export CURRENT_PROJECT_ID='your_project'")
    print("  2. Enable S3 mount: export S3_MOUNT_ENABLED='true'")
    print(f"  3. Configure mount path: export S3_MOUNT_DIR='{configured_mount_dir}'")
    print("  4. BaseDataToolkit will auto-create: $S3_MOUNT_DIR/your_project/{toolkit_name}/")
    print("  5. AI analysis results go to: $S3_MOUNT_DIR/your_project/results/")
    print("  6. All data syncs automatically across environments")
    print("  7. Environment variable ensures consistency across all components")
    
    # Troubleshooting
    if passed < total:
        print("\n🔧 Troubleshooting:")
        if not globals().get('mount_success', False):
            print("  📁 Mount issues: E2B template lacks proper S3 mount or AWS credentials")
            print("    ◦ Root cause: Template built without AWS build arguments")
            print("    ◦ Solution: Rebuild template with: ./setup.sh --e2b-only")
        if not globals().get('structure_success', False):
            print("  🗂️  Structure issues: Check write permissions to mount directory")
        if not globals().get('analysis_success', False):
            print("  📊 Analysis issues: Verify pandas/numpy are available in E2B template")
        if not globals().get('crypto_agent_success', False):
            print("  🤖 Agent issues: Check crypto toolkit imports and E2B agent configuration")
        
        print("\n  🛠️  Root Cause & Fix:")
        print("    ❌ PROBLEM: E2B template was built without AWS credentials as build arguments")
        print("    ✅ SOLUTION: Rebuild E2B template with proper AWS credentials:")
        print("       cd docker/e2b-sandbox")
        print("       e2b template build \\")
        print("         --build-arg AWS_ACCESS_KEY_ID='your_key' \\")
        print("         --build-arg AWS_SECRET_ACCESS_KEY='your_secret' \\")
        print("         --build-arg S3_BUCKET_NAME='your_bucket' \\")
        print(f"         --build-arg S3_MOUNT_DIR='{configured_mount_dir}' \\")
        print("         --name sentient-e2b-s3 --force")
        print()
        print("    Or run automated fix: ./setup.sh --e2b-only")
        
        # Show temporary workaround if available
        if 'discovered_mount_dir' in globals():
            print(f"\n  🔧 TEMPORARY WORKAROUND:")
            print(f"    Update .env with working mount: S3_MOUNT_DIR={globals()['discovered_mount_dir']}")

print_test_summary()

## 🚀 Next Steps

### ✅ If all tests passed:
Your unified mount directory system is working perfectly! You can now:

1. **Use in production**: Data saved in any environment will be visible in all others
2. **Run AI agents**: They will automatically use the correct project structure
3. **Scale across environments**: Local development → Docker deployment → E2B execution

### ⚠️ If some tests failed:
1. **Check the error messages** in the test outputs above
2. **Verify E2B template**: Ensure `/data/sentient` is properly mounted
3. **Check S3 setup**: Verify AWS credentials and bucket permissions
4. **Rebuild template**: `cd docker/e2b-sandbox && e2b template build -n sentient-e2b-s3`

### 🎯 Production Usage:

```bash
# Set your project context
export CURRENT_PROJECT_ID="crypto_analysis_2024"
export S3_MOUNT_ENABLED="true"

# Run your agents - they'll automatically create:
# /data/sentient/crypto_analysis_2024/binance_toolkit/
# /data/sentient/crypto_analysis_2024/coingecko_toolkit/
# /data/sentient/crypto_analysis_2024/results/
```

### 📁 Expected Final Structure:
```
/data/sentient/
├── crypto_analysis_2024/
│   ├── binance_toolkit/          # Real API data
│   │   ├── BTCUSDT_data.parquet
│   │   └── current_prices.json
│   ├── coingecko_toolkit/        # Market data
│   ├── results/                  # AI analysis outputs
│   │   ├── market_analysis.json
│   │   ├── trading_signals.csv
│   │   └── risk_assessment.png
│   └── ...
└── other_projects/
```

### 🎉 Benefits Achieved:
- ✅ **Unified file paths** across all environments
- ✅ **Automatic data sync** via S3 mount
- ✅ **Project isolation** with clear directory structure
- ✅ **Cross-environment persistence** for AI workflows
- ✅ **Scalable architecture** for production deployment